In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append("../../")

In [4]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 12.44 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41325,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 12.44 GiB
Comm: tcp://127.0.0.1:40937,Total threads: 4
Dashboard: http://127.0.0.1:39971/status,Memory: 3.11 GiB
Nanny: tcp://127.0.0.1:46231,


In [94]:
import pipit as pp

In [95]:
trace = pp.Trace.from_otf2("/home/rakrish/otf2-mpi-only-traces/kripke/32/")

In [96]:
df = trace.events
df

,Timestamp (ns),Event Type,Name,Thread,Process,Attributes
npartitions=1,,,,,,
0,float64,category[known],category[known],category[known],category[known],object
11854,...,...,...,...,...,...


In [97]:
df["Process"] = df["Process"].astype("int")
df = df.set_index("Process")
df = df.repartition(npartitions=64)

In [98]:
df

,Timestamp (ns),Event Type,Name,Thread,Attributes
npartitions=31,,,,,
0,float64,category[known],category[known],category[known],object
1,...,...,...,...,...
...,...,...,...,...,...
30,...,...,...,...,...
31,...,...,...,...,...


In [ ]:
import modin.pandas as pd
import numpy as np
df = pd.concat([pd.DataFrame(np.random.randint(0, 100, size=(2**20, 2**8))) for _ in range(40)]) # 40x2GB frames -- Working!
df.info()

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

2023-11-12 14:35:03,060	INFO worker.py:1553 -- Started a local Ray instance.
